In [2]:
using ITensors
using Distributions
using HDF5
using Random
using Plots
using Plots.PlotMeasures
using MLDataUtils, StatsBase
using PyCall
using DelimitedFiles
include("SweepOptRegular.jl");

In [5]:
ecg_dat = readdlm("ECG200_TRAIN.txt")
X_train = ecg_dat[:, 2:end]
y_train = Int.(ecg_dat[:, 1])
remap = Dict(-1 => 0, 1 => 1)
y_train = [remap[label] for label in y_train];
y_train = reshape(y_train, length(y_train), 1);

In [6]:
ecg_dat_test = readdlm("ECG200_TEST.txt")
X_test = ecg_dat_test[:, 2:end]
y_test = Int.(ecg_dat_test[:, 1])
y_test = [remap[label] for label in y_test]
y_test = reshape(y_test, length(y_test), 1);

In [7]:
struct RobustSigmoidTransform{T<:Real} <: AbstractDataTransform
    median::T
    iqr::T
    k::T
    positive::Bool

    function RobustSigmoidTransform(median::T, iqr::T, k::T, positive=true) where T<:Real
        new{T}(median, iqr, k, positive)
    end
end

function robust_sigmoid(x::Real, median::Real, iqr::Real, k::Real, positive::Bool)
    xhat = 1.0 / (1.0 + exp(-(x - median) / (iqr / k)))
    if !positive
        xhat = 2*xhat - 1
    end
    return xhat
end

function fitScaler(::Type{RobustSigmoidTransform}, X::Matrix; k::Real=1.35, positive::Bool=true)
    medianX = median(X)
    iqrX = iqr(X)
    return RobustSigmoidTransform(medianX, iqrX, k, positive)
end

function transformData(t::RobustSigmoidTransform, X::Matrix)
    return map(x -> robust_sigmoid(x, t.median, t.iqr, t.k, t.positive), X)
end

# New SigmoidTransform
struct SigmoidTransform <: AbstractDataTransform
    positive::Bool
end

function sigmoid(x::Real, positive::Bool)
    xhat = 1.0 / (1.0 + exp(-x))
    if !positive
        xhat = 2*xhat - 1
    end
    return xhat
end

function fitScaler(::Type{SigmoidTransform}, X::Matrix; positive::Bool=true)
    return SigmoidTransform(positive)
end

function transformData(t::SigmoidTransform, X::Matrix)
    return map(x -> sigmoid(x, t.positive), X)
end;

In [8]:
rst = fitScaler(RobustSigmoidTransform, X_train; positive=true)
X_train_normalised = transformData(rst, X_train)
X_test_normalised = transformData(rst, X_test);

# Encode as Product State

In [10]:
function AngleEncoder(x::Float64) 
    if x <= 1 && x >= 0
        return [cos(π/2 * x), sin(π/2 * x)]
    else
        println("Data points must be rescaled between 1 and 0 before encoding using the half angle encoder.")
    end
end;

In [13]:
s = siteinds("S=1/2", 96);

In [20]:
function GenerateProductState(normalised_sample::Vector, s)

    """Convert a single normalised sample to a product state with local dimension defined by the feature map."""

    site_inds = s
    product_state = MPS(site_inds; linkdims=1)
    N_sites = length(s)

    if N_sites !== size(normalised_sample)[1]
        error("Number of MPS sites (N = $N_sites) does not match the length of the time-series sample (N = $(size(normalised_sample)[1])).")
    end

    for j=1:N_sites
        T = ITensor(site_inds[j])
        mapped_vals = AngleEncoder(normalised_sample[j])
        up_val, down_val = mapped_vals
        T[1] = up_val
        T[2] = down_val
        product_state[j] = T
    end
    return product_state
end;

In [34]:
class_0_idxs = findall(x -> x.== 0, y_train[:, 1])
class_1_idxs = findall(x -> x.== 1, y_train[:, 1]);

In [51]:
test1 = GenerateProductState(X_train_normalised[class_0_idxs[1],:], s)
test2 = GenerateProductState(X_train_normalised[class_1_idxs[10],:], s)

MPS
[1] ((dim=2|id=357|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=535|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=589|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=262|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=671|"S=1/2,Site,n=5"),)
[6] ((dim=2|id=759|"S=1/2,Site,n=6"),)
[7] ((dim=2|id=431|"S=1/2,Site,n=7"),)
[8] ((dim=2|id=777|"S=1/2,Site,n=8"),)
[9] ((dim=2|id=102|"S=1/2,Site,n=9"),)
[10] ((dim=2|id=469|"S=1/2,Site,n=10"),)
[11] ((dim=2|id=166|"S=1/2,Site,n=11"),)
[12] ((dim=2|id=187|"S=1/2,Site,n=12"),)
[13] ((dim=2|id=167|"S=1/2,Site,n=13"),)
[14] ((dim=2|id=312|"S=1/2,Site,n=14"),)
[15] ((dim=2|id=30|"S=1/2,Site,n=15"),)
[16] ((dim=2|id=57|"S=1/2,Site,n=16"),)
[17] ((dim=2|id=908|"S=1/2,Site,n=17"),)
[18] ((dim=2|id=260|"S=1/2,Site,n=18"),)
[19] ((dim=2|id=928|"S=1/2,Site,n=19"),)
[20] ((dim=2|id=789|"S=1/2,Site,n=20"),)
[21] ((dim=2|id=738|"S=1/2,Site,n=21"),)
[22] ((dim=2|id=249|"S=1/2,Site,n=22"),)
[23] ((dim=2|id=426|"S=1/2,Site,n=23"),)
[24] ((dim=2|id=45|"S=1/2,Site,n=24"),)
[25] ((dim=2|id=659|"S=1/2,Site,n

In [52]:
inner(test1, test2)

0.0031505534093837083

In [80]:
function RescaledLogarithmicFidelity(x1, x2; β=10.0)
    regular_inner_product = inner(x1, x2)
    fidelity = abs(regular_inner_product)
    log10_fidelity = log(10, fidelity)
    rlf = β^(log10_fidelity)
    return rlf
end

RescaledLogarithmicFidelity (generic function with 1 method)

Check that RLF returns regular fidelity when $\beta = 10$

In [67]:
println("RLF $(RescaledLogarithmicFidelity(test1, test2))")
println("Fidelity: $(abs(inner(test1, test2)))")

RLF 0.00315055340938371
Fidelity: 0.0031505534093837083


Try different values of $\beta$

In [79]:
RescaledLogarithmicFidelity(test1, test2; β=8)

0.005505771832382478